In [1]:
#Importing Library

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
#Define Neural Networks Model.

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(784, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 64)
        self.fc5 = nn.Linear(64, 32)
        self.fc6 = nn.Linear(32, 10)

    def forward(self, x):
        x = x.float()
        h1 = F.relu(self.fc1(x.view(-1, 784)))
        h2 = F.relu(self.fc2(h1))
        h3 = F.relu(self.fc3(h2))
        h4 = F.relu(self.fc4(h3))
        h5 = F.relu(self.fc5(h4))
        h6 = self.fc6(h5)
        return F.log_softmax(h6, dim=1)

print("init model done")

init model done


In [3]:
# Set Hyper parameters and other variables to train the model.

batch_size = 64
test_batch_size = 1000
epochs = 10
lr = 0.01
momentum = 0.5
no_cuda = True
seed = 1
log_interval = 200

use_cuda = not no_cuda and torch.cuda.is_available()
torch.manual_seed(seed)
device = torch.device("cuda" if use_cuda else "cpu")
kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
print("set vars and device done")

set vars and device done


In [4]:
#Prepare Data Loader for Training and Validation

transform = transforms.Compose([
                 transforms.ToTensor(),
                 transforms.Normalize((0.1307,), (0.3081,))])

train_loader = torch.utils.data.DataLoader(
  datasets.MNIST('../data', train=True, download=True, 
                 transform=transform), 
    batch_size = batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, download=True,
                 transform=transform), 
    batch_size=test_batch_size, shuffle=True, **kwargs)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz to ../data\MNIST\raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../data\MNIST\raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../data\MNIST\raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\MNIST\raw



C:\Users\Lab01\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:498: UserWarning: The given NumPy array is not writable, and PyTorch does not support non-writable tensors. This means writing to this tensor will result in undefined behavior. You may want to copy the array to protect its data or make it writable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\torch\csrc\utils\tensor_numpy.cpp:178.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [5]:
test_loader

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

In [7]:

def train(log_interval, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test(log_interval, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() 
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format
          (test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [8]:
# Train and Test the model and save it.

for epoch in range(1, 11):
    train(log_interval, model, device, train_loader, optimizer, epoch)
    test(log_interval, model, device, test_loader)
torch.save(model, './model.pt')








Train Epoch: 1 [0/60000 (0%)]	Loss: 2.290312
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2.296319
Train Epoch: 1 [25600/60000 (43%)]	Loss: 2.270592
Train Epoch: 1 [38400/60000 (64%)]	Loss: 2.144830
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1.135325

Test set: Average loss: 0.7305, Accuracy: 7528/10000 (75%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.834934
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.456183
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.357719
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.219388
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.499891

Test set: Average loss: 0.3347, Accuracy: 8987/10000 (90%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 0.706979
Train Epoch: 3 [12800/60000 (21%)]	Loss: 0.092980
Train Epoch: 3 [25600/60000 (43%)]	Loss: 0.265400
Train Epoch: 3 [38400/60000 (64%)]	Loss: 0.203220
Train Epoch: 3 [51200/60000 (85%)]	Loss: 0.204935

Test set: Average loss: 0.1754, Accuracy: 9502/10000 (95%)

Train Epoch: 4 [0/60000 (0%)]	Loss: 0.153128
Train Epoch: 4 [12800/60000 (21%)]	Lo

KeyboardInterrupt: 

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [10]:
class DNN(nn.Module):
    def __init__(self):
        super(DNN, self).__init__()	# 필수
        self.w1 = nn.Linear(2, 20)	# Linear한 1차원 값의 행렬곱을 할 때 사용 한다.
        self.w2 = nn.Linear(20, 3)  # y = xA + b, bias 값을 가지고 있다.bias 비활성화를 원하면 bias=False를 파라미터에 입력 하면 된다.
        self.relu = nn.ReLU()	# ReLU 활성화 함수
        self.softmax = nn.Softmax(dim=0)	# Softmax 함수
        self.dropout = nn.Dropout(0.5)	# Dropout을 위함

    def forward(self, x):	# 값을 도출하는 함수
        y = self.w1(x) 
        y = self.relu(y)
        y = self.dropout(y)
        y = self.w2(y)
        y = self.softmax(y)
        return y

In [11]:
model = DNN() # 객체 선언
criterion = torch.nn.CrossEntropyLoss()  # loss function 설정
optim = torch.optim.Adam(model.parameters(), lr=0.01) # Optimizer 설정

x_data = torch.Tensor([
    [0, 0],
    [1, 0],
    [1, 1],
    [0, 0],
    [0, 0],
    [0, 1]
])

y_data = torch.LongTensor([
    0,  # etc
    1,  # mammal
    2,  # birds
    0,
    0,
    2
])

for epoch in range(1000):
    output = model(x_data)	# model의 forward 함수 호출
    loss = criterion(output, y_data)	# loss function으로 값 계산

    optim.zero_grad()	# 변화도를 0으로 만듦
    loss.backward() # 역전파
    optim.step()	# 역전파로 알아낸 변화도를 model에 적용

    print("progress:", epoch, "loss=", loss.item())

progress: 0 loss= 1.099349856376648
progress: 1 loss= 1.0987968444824219
progress: 2 loss= 1.0830432176589966
progress: 3 loss= 1.0973070859909058
progress: 4 loss= 1.098348617553711
progress: 5 loss= 1.087414264678955
progress: 6 loss= 1.0835736989974976
progress: 7 loss= 1.0824517011642456
progress: 8 loss= 1.0739136934280396
progress: 9 loss= 1.0777336359024048
progress: 10 loss= 1.0680445432662964
progress: 11 loss= 1.0630570650100708
progress: 12 loss= 1.0695565938949585
progress: 13 loss= 1.0727533102035522
progress: 14 loss= 1.0531425476074219
progress: 15 loss= 1.0457011461257935
progress: 16 loss= 1.0489147901535034
progress: 17 loss= 1.0684738159179688
progress: 18 loss= 1.0519918203353882
progress: 19 loss= 1.031972885131836
progress: 20 loss= 0.99236661195755
progress: 21 loss= 1.0343903303146362
progress: 22 loss= 0.9973047375679016
progress: 23 loss= 1.02861750125885
progress: 24 loss= 1.0058774948120117
progress: 25 loss= 0.96169114112854
progress: 26 loss= 1.01510667800

progress: 238 loss= 0.8306422233581543
progress: 239 loss= 0.8321802616119385
progress: 240 loss= 0.8426918983459473
progress: 241 loss= 0.8331936001777649
progress: 242 loss= 0.8350458145141602
progress: 243 loss= 0.8075249791145325
progress: 244 loss= 0.8188349604606628
progress: 245 loss= 0.8110007643699646
progress: 246 loss= 0.8258845210075378
progress: 247 loss= 0.8165770173072815
progress: 248 loss= 0.8191463351249695
progress: 249 loss= 0.8140792846679688
progress: 250 loss= 0.8095300197601318
progress: 251 loss= 0.8160898685455322
progress: 252 loss= 0.8156020641326904
progress: 253 loss= 0.8147860169410706
progress: 254 loss= 0.8191113471984863
progress: 255 loss= 0.8204441070556641
progress: 256 loss= 0.8193013072013855
progress: 257 loss= 0.8272650837898254
progress: 258 loss= 0.8830111622810364
progress: 259 loss= 0.8125896453857422
progress: 260 loss= 0.8472903370857239
progress: 261 loss= 0.8318063616752625
progress: 262 loss= 0.8102491497993469
progress: 263 loss= 0.835

progress: 479 loss= 0.8132009506225586
progress: 480 loss= 0.8087003231048584
progress: 481 loss= 0.8294863104820251
progress: 482 loss= 0.8146526217460632
progress: 483 loss= 0.8097813129425049
progress: 484 loss= 0.8612843155860901
progress: 485 loss= 0.8105485439300537
progress: 486 loss= 0.8226720690727234
progress: 487 loss= 0.8163345456123352
progress: 488 loss= 0.8140185475349426
progress: 489 loss= 0.816623866558075
progress: 490 loss= 0.8180467486381531
progress: 491 loss= 0.8128347992897034
progress: 492 loss= 0.8205134868621826
progress: 493 loss= 0.8148625493049622
progress: 494 loss= 0.834536612033844
progress: 495 loss= 0.8092861771583557
progress: 496 loss= 0.8360483646392822
progress: 497 loss= 0.8096466660499573
progress: 498 loss= 0.8114015460014343
progress: 499 loss= 0.814224898815155
progress: 500 loss= 0.8126316666603088
progress: 501 loss= 0.8099679350852966
progress: 502 loss= 0.811576783657074
progress: 503 loss= 0.8359418511390686
progress: 504 loss= 0.8158421

progress: 703 loss= 0.8070793747901917
progress: 704 loss= 0.8160482048988342
progress: 705 loss= 0.819236695766449
progress: 706 loss= 0.8168098330497742
progress: 707 loss= 0.8049386143684387
progress: 708 loss= 0.80973881483078
progress: 709 loss= 0.8137853741645813
progress: 710 loss= 0.8164191842079163
progress: 711 loss= 0.8162963390350342
progress: 712 loss= 0.8193699717521667
progress: 713 loss= 0.8311119675636292
progress: 714 loss= 0.8165950775146484
progress: 715 loss= 0.808933675289154
progress: 716 loss= 0.813173770904541
progress: 717 loss= 0.8116111159324646
progress: 718 loss= 0.8158835768699646
progress: 719 loss= 0.8879370093345642
progress: 720 loss= 0.8074855804443359
progress: 721 loss= 0.8050721287727356
progress: 722 loss= 0.8037724494934082
progress: 723 loss= 0.8181822896003723
progress: 724 loss= 0.8187036514282227
progress: 725 loss= 0.8165289759635925
progress: 726 loss= 0.8051047325134277
progress: 727 loss= 0.8514611721038818
progress: 728 loss= 0.82604670

progress: 943 loss= 0.8240095973014832
progress: 944 loss= 0.8109709620475769
progress: 945 loss= 0.809380054473877
progress: 946 loss= 0.8126338124275208
progress: 947 loss= 0.8112159371376038
progress: 948 loss= 0.8108217120170593
progress: 949 loss= 0.8072395920753479
progress: 950 loss= 0.8030462861061096
progress: 951 loss= 0.8105799555778503
progress: 952 loss= 0.8134665489196777
progress: 953 loss= 0.8243785500526428
progress: 954 loss= 0.8039471507072449
progress: 955 loss= 0.8095330595970154
progress: 956 loss= 0.8149022459983826
progress: 957 loss= 0.8183448910713196
progress: 958 loss= 0.8074617385864258
progress: 959 loss= 0.8135812282562256
progress: 960 loss= 0.8102639317512512
progress: 961 loss= 0.8188767433166504
progress: 962 loss= 0.813169002532959
progress: 963 loss= 0.811373233795166
progress: 964 loss= 0.8093813061714172
progress: 965 loss= 0.8104636073112488
progress: 966 loss= 0.8042767643928528
progress: 967 loss= 0.8143842816352844
progress: 968 loss= 0.851886

In [12]:
for x in x_data:
    y_pred = model(x)	# 결과
    print(y_pred.max(dim=0)[1].item())	# Softmax로 나온 최고 값의 Item 반환

0
1
2
0
0
2


In [13]:
type(x_data)

torch.Tensor

In [14]:
x_data

tensor([[0., 0.],
        [1., 0.],
        [1., 1.],
        [0., 0.],
        [0., 0.],
        [0., 1.]])

In [17]:
y_data

tensor([0, 1, 2, 0, 0, 2])